In [39]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
import re

def setup_firefox_driver():
    """配置Firefox浏览器驱动"""
    # 设置Firefox选项
    firefox_options = Options()
    # 取消下面的注释可以使Firefox在无头模式下运行（不显示浏览器窗口）
    # firefox_options.add_argument("--headless")
    
    # 创建Firefox驱动实例
    driver = webdriver.Firefox(options=firefox_options)
    
    # 设置隐式等待时间
    driver.implicitly_wait(10)
    
    return driver

def scrape_esf_listings(driver, url, pages=11):
    """爬取房天下二手房信息"""
    all_listings = []
    
    try:
        driver.get(url)
        print(f"正在访问: {url}")
        time.sleep(5)  # 增加等待时间，确保页面完全加载
        
        for page in range(1, pages + 1):
            print(f"正在爬取第 {page} 页...")
            
            # 等待房源列表加载
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.shop_list'))
                )
            except TimeoutException:
                print("等待房源列表加载超时，尝试继续...")
            
            # 获取所有房源元素
            listings = driver.find_elements(By.CSS_SELECTOR, '.shop_list dl')
            
            for listing in listings:
                try:
                    # 提取标题
                    try:
                        title_element = listing.find_element(By.CSS_SELECTOR, ".tit_shop")
                        title = title_element.text.strip()
                    except:
                        title = "未找到标题"
                    
                    # 提取tel_shop中的信息
                    house_info = {}
                    try:
                        tel_shop = listing.find_element(By.CSS_SELECTOR, "p.tel_shop")
                        tel_content = tel_shop.text
                        
                        # 按行分割内容
                        lines = tel_content.strip().split('|')
                        
                        
                        # 提取户型
                        house_type = lines[0].strip() if len(lines) > 0 else "未知"
                        house_info["户型"] = house_type
                        
                        # 提取面积
                        area = lines[1].strip() if len(lines) > 1 else "未知"
                        house_info["面积"] = area
                        
                        # 提取楼层信息（如果有）
                        floor_info = lines[2].strip() if len(lines) > 2 else "未知"
                        house_info["楼层"] = floor_info
                        
                        # 提取朝向
                        orientation = lines[3].strip() if len(lines) > 3 else "未知"
                        house_info["朝向"] = orientation
                        
                        # 提取建筑年份
                        year_built = lines[4].strip() if len(lines) > 4 else "未知"
                        house_info["建筑年份"] = year_built
                        
                    except Exception as e:
                        print(f"提取tel_shop信息时出错: {e}")
                        house_info = {
                            "户型": "未知",
                            "面积": "未知",
                            "楼层": "未知",
                            "朝向": "未知",
                            "建筑年份": "未知"
                        }
                    
                    # 提取地址
                    try:
                        address_element = listing.find_element(By.CSS_SELECTOR, "p.add_shop")
                        address = address_element.text.strip()
                    except:
                        address = "未知"
                    
                    # 提取价格信息
                    try:
                        # 总价
                        price_element = listing.find_element(By.CSS_SELECTOR, "dd.price_right span.red b")
                        total_price = price_element.text.strip()
                        
                        # 单价
                        unit_price_element = listing.find_element(By.CSS_SELECTOR, "dd.price_right span:not(.red)")
                        unit_price = unit_price_element.text.strip()
                        
                    except Exception as e:
                        print(f"提取价格信息时出错: {e}")
                        total_price = "未知"
                        unit_price = "未知"
                    
                    # 整理数据
                    listing_data = {
                        "标题": title,
                        "户型": house_info["户型"],
                        "面积": house_info["面积"],
                        "楼层": house_info["楼层"],
                        "朝向": house_info["朝向"],
                        "建筑年份": house_info["建筑年份"],
                        "地址": address,
                        "总价": total_price,
                        "单价": unit_price
                    }
                    
                    all_listings.append(listing_data)
                    print(f"已爬取: {title}")
                    
                except Exception as e:
                    print(f"提取房源信息时出错: {e}")
            
            # 检查是否有下一页并点击 (如果当前页小于总页数)
            if page < pages:
                try:
                    # 确保滚动到页面底部
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    
                    # 尝试多种方式定位下一页按钮
                    next_page = None
                    
                    # 方法1: 通过XPath定位下一页文本
                    try:
                        next_page = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '下一页')]"))
                        )
                    except TimeoutException:
                        print("通过文本定位下一页按钮失败，尝试其他方法...")
                    
                    # 方法2: 通过CSS选择器尝试定位分页控件中的下一页按钮
                    if not next_page:
                        try:
                            next_page = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.CSS_SELECTOR, ".fanye a.next"))
                            )
                        except TimeoutException:
                            print("通过CSS选择器定位下一页按钮失败，尝试其他方法...")
                    
                    # 方法3: 尝试通过当前页码定位下一页
                    if not next_page:
                        try:
                            current_page_element = driver.find_element(By.CSS_SELECTOR, ".fanye span.on")
                            current_page = int(current_page_element.text)
                            next_page_selector = f".fanye a[href*='i3{current_page+1}']"
                            next_page = driver.find_element(By.CSS_SELECTOR, next_page_selector)
                        except (NoSuchElementException, ValueError) as e:
                            print(f"通过页码定位下一页按钮失败: {e}")
                    
                    # 如果找到了下一页按钮，使用JavaScript点击它
                    if next_page:
                        print("找到下一页按钮，准备点击...")
                        # 使用JS点击，避免可能的覆盖问题
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_page)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", next_page)
                        print(f"已点击下一页按钮，即将加载第 {page+1} 页")
                        time.sleep(5)  # 增加等待时间，确保新页面加载完成
                    else:
                        print("未能找到下一页按钮，可能已到最后一页")
                        break
                        
                except Exception as e:
                    print(f"翻页过程中出错: {e}")
                    # 捕获当前页面源代码和截图，用于调试
                    with open(f"page_{page}_source.html", "w", encoding="utf-8") as f:
                        f.write(driver.page_source)
                    driver.save_screenshot(f"page_{page}_screenshot.png")
                    print(f"已保存当前页面源代码和截图")
                    break
    
    except Exception as e:
        print(f"爬虫运行时出错: {e}")
    
    return all_listings

def save_to_csv(data, filename="房天下二手房数据.csv"):
    """将爬取的数据保存为CSV文件"""
    if not data:
        print("没有数据可保存")
        return
    
    keys = data[0].keys()
    
    with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"数据已保存到 {filename}")

def main():
    url = "https://esf.fang.com/house-a010-b05048/"
    driver = setup_firefox_driver()
    
    try:
        # 设置窗口大小，确保页面元素可见
        driver.maximize_window()
        
        listings = scrape_esf_listings(driver, url, pages=11)  # 爬取11页
        save_to_csv(listings)
    finally:
        driver.quit()
        print("浏览器已关闭")

if __name__ == "__main__":
    main()

正在访问: https://esf.fang.com/house-a010-b05048/
正在爬取第 1 页...
已爬取: 新房超值豪宅 完美户型 业主诚售,新政省税5万!
已爬取: 亦庄橡树湾 3室2厅 93.29平
已爬取: 合生世界花园南北朝向3居3卫带靠谱出售
已爬取: 亦庄金悦郡 270飘窗 三面宽三居室 南北通透 看房方便
已爬取: 亦庄橡树湾 3室2厅 94.4平
已爬取: 马驹桥 样本小区 南北通透三居 大横厅 南向大阳台
已爬取: “合生世界村(E区)”让您以看风景的心情来欣赏家
已爬取: (免 ,用,金)三居两卫户型,亦庄,新房花园洋房,精装准现房
已爬取: 亦庄橡树湾 3室1厅 93.66平
已爬取: 亦庄橡树湾 4室2厅 141.2平
已爬取: 给我一个电话,帮您找个温馨的家!优山美地,不容错过!
已爬取: 珠江逸景家园(南区) 精装2居室 观景房
已爬取: 新海南里 精装1居室 观景房
已爬取: 东亚瑞晶苑正规2居室婚房装修采光好空间大不临街
已爬取: 宏仁家园|2室2厅1卫1阳台|85.88平米|西|14层
已爬取: 珠江逸景家园(南区) 精装2居室 观景房
已爬取: 宏仁家园高层观景 2居 户型方正 亲自看过此房
已爬取: 给我一个电话,帮您找个温馨的家!优山美地,不容错过!
已爬取: 重点推荐金地格林小镇6,不一样的房子 不一样的选择!!
已爬取: 我写的不是标题和描述,写的是你们没有找到一套好的房源的遗憾
已爬取: 样本超值南北 可遇不可求 温馨6室
已爬取: 橡树湾南区 南北 两居室 高端品质社区 南北稀有户型
已爬取: 马驹桥 · 亦庄橡树湾北区 · 3室 · 1厅
已爬取: 新海北里 南北通透两居 满五 诚心出售
已爬取: 全明格局!!户型方正!!业主急出售大降价!!
已爬取: “融科钧廷”让您以看风景的心情来欣赏家
已爬取: 楼层好,视野广,珠江逸景家园(南区)圆您置业梦想!
已爬取: 亦庄新城,宏仁家园南向两居室,不临街,中心位置,随时可以看
已爬取: D调的奢华:明发滨江,欧式田园风,豪华装修
已爬取: 新海北里|2室1厅1卫1阳台|66.76平米|南北|5层
已爬取: 金悦郡 三居室 三面宽 双通透户型 精装修 新房无
已爬取: 合生世界花园电梯房正规1居室带独立客厅出行方便有钥匙
已爬取: 楼层好,视野广,融科香